In [1]:
from confluent_kafka import Producer
import json
import random
from datetime import datetime, timedelta

# Конфигурация Kafka Producer
producer_config = {
    'bootstrap.servers': 'localhost:9092',  # Адрес брокера Kafka
    'client.id': 'transaction-producer'
}

producer = Producer(producer_config)

# Функция для генерации случайной транзакции
def generate_transaction(user_id):
    transaction_id = random.randint(1000, 9999)
    amount = round(random.uniform(50, 2000), 2)
    timestamp = datetime.now() - timedelta(
        days=random.randint(0, 30),
        hours=random.randint(0, 23),
        minutes=random.randint(0, 59)
    )
    location = random.choice(["New York", "Los Angeles", "Chicago", "Houston", "Phoenix"])

    return {
        "transaction_id": transaction_id,
        "user_id": user_id,
        "amount": amount,
        "timestamp": timestamp.isoformat(),
        "location": location
    }

# Генерация и отправка транзакций
topic = 'transactions'

for user_id in range(1, 101):  # Для 100 пользователей
    for _ in range(10):  # Каждому пользователю по 10 транзакций
        transaction = generate_transaction(user_id)
        producer.produce(topic, key=str(user_id), value=json.dumps(transaction))
        print(f"Отправлена транзакция: {transaction}")

producer.flush()


Отправлена транзакция: {'transaction_id': 8069, 'user_id': 1, 'amount': 1599.86, 'timestamp': '2024-11-26T20:23:25.487069', 'location': 'Chicago'}
Отправлена транзакция: {'transaction_id': 2625, 'user_id': 1, 'amount': 741.86, 'timestamp': '2024-12-11T03:28:25.487069', 'location': 'New York'}
Отправлена транзакция: {'transaction_id': 2114, 'user_id': 1, 'amount': 619.04, 'timestamp': '2024-12-04T07:07:25.487069', 'location': 'Chicago'}
Отправлена транзакция: {'transaction_id': 2916, 'user_id': 1, 'amount': 612.46, 'timestamp': '2024-11-25T07:13:25.487069', 'location': 'Los Angeles'}
Отправлена транзакция: {'transaction_id': 8247, 'user_id': 1, 'amount': 1690.46, 'timestamp': '2024-12-01T20:09:25.487069', 'location': 'New York'}
Отправлена транзакция: {'transaction_id': 2677, 'user_id': 1, 'amount': 592.51, 'timestamp': '2024-12-06T17:47:25.487069', 'location': 'New York'}
Отправлена транзакция: {'transaction_id': 2179, 'user_id': 1, 'amount': 1572.08, 'timestamp': '2024-12-17T02:10:25.

0

In [2]:
import sqlite3

# Создаем базу данных и таблицу клиентов
conn = sqlite3.connect('banking_app.db')
cursor = conn.cursor()

# Создание таблицы клиентов
cursor.execute('''
CREATE TABLE IF NOT EXISTS clients (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL,
    email TEXT NOT NULL,
    phone TEXT NOT NULL
)
''')

# Генерация данных клиентов
names = ["Alice", "Bob", "Charlie", "David", "Eve"]
for i in range(1, 101):
    cursor.execute('''
    INSERT INTO clients (name, email, phone) VALUES (?, ?, ?)
    ''', (random.choice(names), f'user{i}@example.com', f'123-456-789{i % 10}'))

conn.commit()
conn.close()


In [3]:
from confluent_kafka import Consumer, KafkaException
import sqlite3
import json

# Конфигурация Kafka Consumer
consumer_config = {
    'bootstrap.servers': 'localhost:9092',  # Адрес брокера Kafka
    'group.id': 'transaction-group',
    'auto.offset.reset': 'earliest'  # Чтение с самого начала
}

consumer = Consumer(consumer_config)
consumer.subscribe(['transactions'])

# Проверка на подозрительность: сумма > 1500
def is_suspicious(transaction):
    return transaction['amount'] > 1500

# Подключение к базе данных
conn = sqlite3.connect('banking_app.db')
cursor = conn.cursor()

suspicious_producer = Producer(producer_config)

# Обработка сообщений и отправка подозрительных транзакций
def send_suspicious_transaction(transaction):
    suspicious_producer.produce('suspicious_transactions', key=str(transaction['user_id']), value=json.dumps(transaction))
    suspicious_producer.flush()
    print(f"Подозрительная транзакция отправлена в 'suspicious_transactions': {transaction}")

# Обработка сообщений
try:
    while True:
        msg = consumer.poll(1.0)  # Ожидание сообщений 1 секунда

        if msg is None:
            continue
        if msg.error():
            if msg.error().code() == KafkaError._PARTITION_EOF:
                continue
            else:
                print(f"Ошибка Kafka: {msg.error()}")
                break

        # Десериализация сообщения
        transaction = json.loads(msg.value().decode('utf-8'))
        user_id = transaction['user_id']

        # Поиск клиента в базе данных
        cursor.execute('SELECT * FROM clients WHERE id = ?', (user_id,))
        client = cursor.fetchone()

        # Проверка на подозрительность
        if client and is_suspicious(transaction):
            print(f"Обнаружена подозрительная транзакция: {transaction} для клиента: {client}")
            send_suspicious_transaction(transaction)

except KeyboardInterrupt:
    print("Остановка консюмера.")
finally:
    consumer.close()
    conn.close()


Обнаружена подозрительная транзакция: {'transaction_id': 8069, 'user_id': 1, 'amount': 1599.86, 'timestamp': '2024-11-26T20:23:25.487069', 'location': 'Chicago'} для клиента: (1, 'David', 'user1@example.com', '123-456-7891')
Подозрительная транзакция отправлена в 'suspicious_transactions': {'transaction_id': 8069, 'user_id': 1, 'amount': 1599.86, 'timestamp': '2024-11-26T20:23:25.487069', 'location': 'Chicago'}
Обнаружена подозрительная транзакция: {'transaction_id': 8247, 'user_id': 1, 'amount': 1690.46, 'timestamp': '2024-12-01T20:09:25.487069', 'location': 'New York'} для клиента: (1, 'David', 'user1@example.com', '123-456-7891')
Подозрительная транзакция отправлена в 'suspicious_transactions': {'transaction_id': 8247, 'user_id': 1, 'amount': 1690.46, 'timestamp': '2024-12-01T20:09:25.487069', 'location': 'New York'}
Обнаружена подозрительная транзакция: {'transaction_id': 2179, 'user_id': 1, 'amount': 1572.08, 'timestamp': '2024-12-17T02:10:25.487069', 'location': 'Chicago'} для кл